In [58]:
import pandas as pd
import datetime

In [8]:
companies_with_two_tickers_flatten = pd.read_csv('companies_with_two_tickers_flatten_2024-04-18_12-50-54.csv', index_col=0)

In [9]:
print(companies_with_two_tickers_flatten.head(10))
print(len(companies_with_two_tickers_flatten))

     cik                             company     ticker gurufocus_stockid  \
15  3153                    ALABAMA POWER CO  ALPpQ.PFD            US0TUZ   
18  3333                 ALBERTSONS INC /DE/        ACI            US280P   
20  3449  ALEXANDER & ALEXANDER SERVICES INC       ALEX            US06BL   
35  4281               HOWMET AEROSPACE INC.        HWM            US0SWN   
39  4447                           HESS CORP        HES            US07TL   
56  5272  AMERICAN INTERNATIONAL GROUP, INC.        AIG            US06AN   
61  5611                            FINA INC        TFC            US06JN   
64  5907                           AT&T CORP          T            US09O4   
91  7332              SOUTHWESTERN ENERGY CO        SWN            US09NC   
97  7789                ASSOCIATED BANC-CORP        ASB            US06FE   

    second_match                             gurufocus-company-name  
15         False  \n    Alabama Power Co  (Delisted)\n    (NYSE:...  
18         F

In [138]:
def clean_gurufocus_company_name_column(col):
    col = col.str.strip()
    col = col.astype(str)
    col = col.apply(lambda x: x.split('\n')[0])
    col = col.str.strip()
    return col

In [61]:
def get_datetime_now():
    return datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [13]:
companies_with_two_tickers_flatten['gurufocus-company-name'] = clean_gurufocus_company_name_column(companies_with_two_tickers_flatten['gurufocus-company-name'])

companies_with_two_tickers_flatten.head(10)

,cik,company,ticker,gurufocus_stockid,second_match,gurufocus-company-name
15,3153,ALABAMA POWER CO,ALPpQ.PFD,US0TUZ,False,Alabama Power Co (Delisted)
18,3333,ALBERTSONS INC /DE/,ACI,US280P,False,Albertsons Companies Inc
20,3449,ALEXANDER & ALEXANDER SERVICES INC,ALEX,US06BL,False,Alexander & Baldwin Inc
35,4281,HOWMET AEROSPACE INC.,HWM,US0SWN,False,Howmet Aerospace Inc
39,4447,HESS CORP,HES,US07TL,False,Hess Corp
56,5272,"AMERICAN INTERNATIONAL GROUP, INC.",AIG,US06AN,False,American International Group Inc
61,5611,FINA INC,TFC,US06JN,False,Truist Financial Corp
64,5907,AT&T CORP,T,US09O4,False,AT&T Inc
91,7332,SOUTHWESTERN ENERGY CO,SWN,US09NC,False,Southwestern Energy Co
97,7789,ASSOCIATED BANC-CORP,ASB,US06FE,False,Associated Banc-Corp


In [ ]:
datetime_now = get_datetime_now()
companies_with_two_tickers_flatten.to_csv(f'companies_with_two_tickers_flatten_{datetime_now}_cleaned.csv')

From raw downloaded html page source, extract the Gurufocus CIK

In [110]:
import os

directory = 'gurufocus-immediate-response-for-stockid-two-tickers-case'

not_found_gurufocus_ciks_two_ticker_case = []

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        
        with open(f'{directory}/{filename}', 'r', encoding='utf-8') as f:
            content = f.read()
            cik_index = content.find('cik')  
            
            if cik_index != -1:
                
                filename = filename.replace('.txt', '')
                filename = filename.replace('page_source-', '')
                cik, gurufocus_stockid = filename.split('_')[0].split('-')
                ticker = filename.split('_')[1]
                
                gurufocus_cik = content[cik_index + 5: cik_index + 15]
                try:
                    gurufocus_cik = int(gurufocus_cik.split('"')[0])
                    mask = (companies_with_two_tickers_flatten['cik'] == int(cik)) & (companies_with_two_tickers_flatten['gurufocus_stockid'] == gurufocus_stockid)
                    companies_with_two_tickers_flatten.loc[mask, 'gurufocus-cik'] = gurufocus_cik
                except:
                    print(f"Error for file: {filename}. Gurufocus cik: {gurufocus_cik}")
                    not_found_gurufocus_ciks_two_ticker_case.append([cik, gurufocus_stockid, ticker])
                
            else:
                print(f"cik not found. File: {filename}")
            
    else:
        print(f"File {filename} is not a TXT file")

Error for file: 1004980-CA03R4_PGN.H. Gurufocus cik: ,class_des
Error for file: 1016933-CA01JL_R. Gurufocus cik: ,class_des
Error for file: 1061068-CA02XP_GBO.H. Gurufocus cik: ,class_des
Error for file: 1129989-CA03R4_PGN.H. Gurufocus cik: ,class_des
Error for file: 1143155-US0ST1_XBKS. Gurufocus cik: ,class_des
Error for file: 1387467-CA00WV_HAP.UN. Gurufocus cik: ,class_des
Error for file: 1393311-CA01I2_PUB. Gurufocus cik: ,class_des
Error for file: 1442741-US0ST1_XBKS. Gurufocus cik: ,class_des
Error for file: 1600125-US04DV_MRDN. Gurufocus cik: ,class_des
Error for file: 1656239-US0STA_PNK. Gurufocus cik: ,class_des
Error for file: 1800227-CA00Z5_IAC. Gurufocus cik: ,class_des
Error for file: 276104-US0AYG_UMDA. Gurufocus cik: ,class_des
Error for file: 320387-US0AI6_FMJE. Gurufocus cik: ,class_des
Error for file: 704054-US0AF2_CZPY. Gurufocus cik: ,class_des
Error for file: 707606-CA04JN_XDS_H. Gurufocus cik: ,class_des
Error for file: 723916-US04DV_MRDN. Gurufocus cik: ,class_d

In [111]:
len(not_found_gurufocus_ciks_two_ticker_case)

23

In [17]:
companies_with_two_tickers_flatten.head(10)

,cik,company,ticker,gurufocus_stockid,second_match,gurufocus-company-name,gurufocus-cik
15,3153,ALABAMA POWER CO,ALPpQ.PFD,US0TUZ,False,Alabama Power Co (Delisted),3153.0
18,3333,ALBERTSONS INC /DE/,ACI,US280P,False,Albertsons Companies Inc,1646972.0
20,3449,ALEXANDER & ALEXANDER SERVICES INC,ALEX,US06BL,False,Alexander & Baldwin Inc,1545654.0
35,4281,HOWMET AEROSPACE INC.,HWM,US0SWN,False,Howmet Aerospace Inc,4281.0
39,4447,HESS CORP,HES,US07TL,False,Hess Corp,4447.0
56,5272,"AMERICAN INTERNATIONAL GROUP, INC.",AIG,US06AN,False,American International Group Inc,5272.0
61,5611,FINA INC,TFC,US06JN,False,Truist Financial Corp,92230.0
64,5907,AT&T CORP,T,US09O4,False,AT&T Inc,732717.0
91,7332,SOUTHWESTERN ENERGY CO,SWN,US09NC,False,Southwestern Energy Co,7332.0
97,7789,ASSOCIATED BANC-CORP,ASB,US06FE,False,Associated Banc-Corp,7789.0


In [18]:
unique_by_cik_and_gurufocus_stockid = companies_with_two_tickers_flatten['cik'].astype(str) + companies_with_two_tickers_flatten['gurufocus_stockid']
print(len(unique_by_cik_and_gurufocus_stockid.unique()))

1318


In [19]:
companies_with_two_tickers_flatten_with_gurufocus_cik = companies_with_two_tickers_flatten[companies_with_two_tickers_flatten['gurufocus-cik'].notnull()]
print(len(companies_with_two_tickers_flatten_with_gurufocus_cik))
print(len(companies_with_two_tickers_flatten))

1222
1318


In [20]:
companies_with_two_tickers_flatten_with_gurufocus_cik['cik-equal-gurufocus-cik'] = companies_with_two_tickers_flatten_with_gurufocus_cik['cik'] == companies_with_two_tickers_flatten_with_gurufocus_cik['gurufocus-cik']

print(companies_with_two_tickers_flatten_with_gurufocus_cik['cik-equal-gurufocus-cik'].value_counts())

cik-equal-gurufocus-cik
True     864
False    358
Name: count, dtype: int64


C:\Users\grzeg\AppData\Local\Temp\ipykernel_12468\1622217342.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  companies_with_two_tickers_flatten_with_gurufocus_cik['cik-equal-gurufocus-cik'] = companies_with_two_tickers_flatten_with_gurufocus_cik['cik'] == companies_with_two_tickers_flatten_with_gurufocus_cik['gurufocus-cik']


In [22]:
companies_with_two_tickers_flatten_with_gurufocus_cik['first_match'] = ~companies_with_two_tickers_flatten_with_gurufocus_cik['second_match']
companies_with_two_tickers_flatten_with_gurufocus_cik.head(10)

C:\Users\grzeg\AppData\Local\Temp\ipykernel_12468\2967057231.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  companies_with_two_tickers_flatten_with_gurufocus_cik['first_match'] = ~companies_with_two_tickers_flatten_with_gurufocus_cik['second_match']


,cik,company,ticker,gurufocus_stockid,second_match,gurufocus-company-name,gurufocus-cik,cik-equal-gurufocus-cik,first_match
15,3153,ALABAMA POWER CO,ALPpQ.PFD,US0TUZ,False,Alabama Power Co (Delisted),3153.0,True,True
18,3333,ALBERTSONS INC /DE/,ACI,US280P,False,Albertsons Companies Inc,1646972.0,False,True
20,3449,ALEXANDER & ALEXANDER SERVICES INC,ALEX,US06BL,False,Alexander & Baldwin Inc,1545654.0,False,True
35,4281,HOWMET AEROSPACE INC.,HWM,US0SWN,False,Howmet Aerospace Inc,4281.0,True,True
39,4447,HESS CORP,HES,US07TL,False,Hess Corp,4447.0,True,True
56,5272,"AMERICAN INTERNATIONAL GROUP, INC.",AIG,US06AN,False,American International Group Inc,5272.0,True,True
61,5611,FINA INC,TFC,US06JN,False,Truist Financial Corp,92230.0,False,True
64,5907,AT&T CORP,T,US09O4,False,AT&T Inc,732717.0,False,True
91,7332,SOUTHWESTERN ENERGY CO,SWN,US09NC,False,Southwestern Energy Co,7332.0,True,True
97,7789,ASSOCIATED BANC-CORP,ASB,US06FE,False,Associated Banc-Corp,7789.0,True,True


In [ ]:
companies_with_two_tickers_flatten_with_gurufocus_cik.sort_values(by=['cik-equal-gurufocus-cik', 'first_match'], inplace=True)

companies_with_two_tickers_flatten_with_gurufocus_cik = companies_with_two_tickers_flatten_with_gurufocus_cik[['company', 'gurufocus-company-name', 'first_match', 'cik', 'ticker', 'gurufocus_stockid', 'gurufocus-cik', 'cik-equal-gurufocus-cik']]

# companies_with_two_tickers_flatten_with_gurufocus_cik.to_csv('companies_with_two_tickers_flatten_with_gurufocus_cik_2024-04-18_12-50-54.csv')

In [40]:
matched_companies_from_first_ticker = companies_with_two_tickers_flatten_with_gurufocus_cik[(companies_with_two_tickers_flatten_with_gurufocus_cik['cik-equal-gurufocus-cik'] == True) & (companies_with_two_tickers_flatten_with_gurufocus_cik['first_match'] == True)]

len(matched_companies_from_first_ticker)

448

In [41]:
not_matched_companies_from_first_ticker = companies_with_two_tickers_flatten_with_gurufocus_cik[(companies_with_two_tickers_flatten_with_gurufocus_cik['cik-equal-gurufocus-cik'] == False) & (companies_with_two_tickers_flatten_with_gurufocus_cik['first_match'] == True)]

print(len(not_matched_companies_from_first_ticker))

175


In [42]:
matched_companies_from_second_ticker = companies_with_two_tickers_flatten_with_gurufocus_cik[(companies_with_two_tickers_flatten_with_gurufocus_cik['cik-equal-gurufocus-cik'] == True) & (companies_with_two_tickers_flatten_with_gurufocus_cik['first_match'] == False)]

print(len(matched_companies_from_second_ticker))

416


In [43]:
# get companies that are in both df's not_matched_companies_from_first_ticker, matched_companies_from_second_ticker
companies_not_matched_by_first_but_matched_by_second_ticker = matched_companies_from_second_ticker[matched_companies_from_second_ticker['cik'].isin(not_matched_companies_from_first_ticker['cik'])]
print(len(companies_not_matched_by_first_but_matched_by_second_ticker))
companies_not_matched_by_first_but_matched_by_second_ticker.head(10)

44


,cik,company,ticker,gurufocus_stockid,second_match,gurufocus-company-name,gurufocus-cik,cik-equal-gurufocus-cik,first_match
20,3449,ALEXANDER & ALEXANDER SERVICES INC,AAL,US066A,True,Alexander & Alexander (Delisted),3449.0,True,False
483,39311,INDEPENDENT BANK CORP /MI/,IBCP,US03RB,True,Independent Bank Corp (Ionia MI),39311.0,True,False
602,50104,ANDEAVOR,ANDV,US09U9,True,Andeavor (Delisted),50104.0,True,False
783,67182,MOBIL CORP,MOB,US08HL,True,Mobil Corp (Delisted),67182.0,True,False
894,75252,OWENS & MINOR INC/VA/,OMI,US08WS,True,Owens & Minor Inc,75252.0,True,False
1594,353944,INTERNATIONAL GAME TECHNOLOGY,IGT,US07ZB,True,International Game Technology (Delisted),353944.0,True,False
1673,702430,CENTENNIAL BANCORP,CEBC,US02K4,True,Centennial Bancorp (Delisted),702430.0,True,False
1775,714710,COMMUNITY BANKS INC /PA/,CMTY,US02PJ,True,Community Banks Inc (Delisted),714710.0,True,False
1781,715096,COMMERCE BANCORP INC /NJ/,CBH,US06S6,True,Commerce Bancorp Inc (Delisted),715096.0,True,False
1842,719402,FIRST NATIONAL CORP /VA/,FXNC,US0HF2,True,First National Corp,719402.0,True,False


In [84]:
companies_with_two_tickers_matched = pd.concat([matched_companies_from_first_ticker, companies_not_matched_by_first_but_matched_by_second_ticker]).reset_index(drop=True)
companies_with_two_tickers_matched['gurufocus-cik'] = companies_with_two_tickers_matched['gurufocus-cik'].astype(int)
companies_with_two_tickers_matched.drop(columns=['second_match', 'gurufocus-cik', 'cik-equal-gurufocus-cik'], inplace=True)
print(len(companies_with_two_tickers_matched))
companies_with_two_tickers_matched.head(10)

492


,cik,company,ticker,gurufocus_stockid,gurufocus-company-name,first_match
0,3153,ALABAMA POWER CO,ALPpQ.PFD,US0TUZ,Alabama Power Co (Delisted),True
1,4281,HOWMET AEROSPACE INC.,HWM,US0SWN,Howmet Aerospace Inc,True
2,4447,HESS CORP,HES,US07TL,Hess Corp,True
3,5272,"AMERICAN INTERNATIONAL GROUP, INC.",AIG,US06AN,American International Group Inc,True
4,7332,SOUTHWESTERN ENERGY CO,SWN,US09NC,Southwestern Energy Co,True
5,7789,ASSOCIATED BANC-CORP,ASB,US06FE,Associated Banc-Corp,True
6,12208,"BIO-RAD LABORATORIES, INC.",BIO,US06MF,Bio-Rad Laboratories Inc,True
7,18349,SYNOVUS FINANCIAL CORP,SNV,US09JB,Synovus Financial Corp,True
8,19617,JPMORGAN CHASE & CO,JPM,US083S,JPMorgan Chase & Co,True
9,22356,COMMERCE BANCSHARES INC /MO/,CBSH,US02IM,Commerce Bancshares Inc,True


In [98]:
companies_with_two_tickers_matched_by_second_ticker = companies_with_two_tickers_matched[companies_with_two_tickers_matched['first_match'] == False]
companies_with_two_tickers_matched_by_second_ticker.head(10)

,cik,company,ticker,gurufocus_stockid,gurufocus-company-name,first_match
448,3449,ALEXANDER & ALEXANDER SERVICES INC,AAL,US066A,Alexander & Alexander (Delisted),False
449,39311,INDEPENDENT BANK CORP /MI/,IBCP,US03RB,Independent Bank Corp (Ionia MI),False
450,50104,ANDEAVOR,ANDV,US09U9,Andeavor (Delisted),False
451,67182,MOBIL CORP,MOB,US08HL,Mobil Corp (Delisted),False
452,75252,OWENS & MINOR INC/VA/,OMI,US08WS,Owens & Minor Inc,False
453,353944,INTERNATIONAL GAME TECHNOLOGY,IGT,US07ZB,International Game Technology (Delisted),False
454,702430,CENTENNIAL BANCORP,CEBC,US02K4,Centennial Bancorp (Delisted),False
455,714710,COMMUNITY BANKS INC /PA/,CMTY,US02PJ,Community Banks Inc (Delisted),False
456,715096,COMMERCE BANCORP INC /NJ/,CBH,US06S6,Commerce Bancorp Inc (Delisted),False
457,719402,FIRST NATIONAL CORP /VA/,FXNC,US0HF2,First National Corp,False


In [62]:
datetime_now = get_datetime_now()
companies_with_two_tickers_matched.to_csv(f'companies_with_two_tickers_matched_{datetime_now}.csv')

In [63]:
prediction_subset = pd.read_csv('ECL_with_ticker_2024-04-14_15-20-33.csv', index_col=0)
len(prediction_subset)

C:\Users\grzeg\AppData\Local\Temp\ipykernel_12468\3995222775.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  prediction_subset = pd.read_csv('ECL_with_ticker_2024-04-14_15-20-33.csv', index_col=0)


84652

In [65]:
prediction_subset.tail(10)

,cik,company,period_of_report,gvkey,datadate,filename,qualified,label,bankruptcy_prediction_split,bankruptcy_date_1,bankruptcy_date_2,bankruptcy_date_3,filing_date,ticker,exchange,gurufocus-stockid,second-match-ticker,second-match-exchange,second-match-gurufocus-stockid,gurufocus-company-name
84642,1796209,API GROUP CORP,2020-12-31,325576.0,31/12/2020,/2020/1796209_10K_2020_0001564590-21-014986.json,Yes,False,test,NaN,NaN,NaN,2021-03-24,APG,NYSE,US0UBH,NaN,NaN,NaN,\n APi Group Corp \n (NYSE:APG)\n
84643,1810739,"RADIUS GLOBAL INFRASTRUCTURE, INC.",2020-12-31,325942.0,31/12/2020,/2020/1810739_10K_2020_0001564590-21-016621.json,Yes,False,test,NaN,NaN,NaN,2021-03-30,RADI,NAS,US28N7,NaN,NaN,NaN,\n Radius Global Infrastructure Inc (Delis...
84644,1720635,NVENT ELECTRIC PLC,2018-12-31,326688.0,31/12/2018,/2018/1720635_10K_2018_0001720635-19-000008.json,Yes,False,test,NaN,NaN,NaN,2019-02-19,NVT,NYSE,US0V7E,NaN,NaN,NaN,\n nVent Electric PLC \n (NYSE:NVT)\n
84645,1720635,NVENT ELECTRIC PLC,2019-12-31,326688.0,31/12/2019,/2019/1720635_10K_2019_0001720635-20-000009.json,Yes,False,test,NaN,NaN,NaN,2020-02-25,NVT,NYSE,US0V7E,NaN,NaN,NaN,\n nVent Electric PLC \n (NYSE:NVT)\n
84646,1720635,NVENT ELECTRIC PLC,2020-12-31,326688.0,31/12/2020,/2020/1720635_10K_2020_0001720635-21-000013.json,Yes,False,test,NaN,NaN,NaN,2021-02-23,NVT,NYSE,US0V7E,NaN,NaN,NaN,\n nVent Electric PLC \n (NYSE:NVT)\n
84647,1739445,"ARCOSA, INC.",2018-12-31,328795.0,31/12/2018,/2018/1739445_10K_2018_0001739445-19-000020.json,Yes,False,test,NaN,NaN,NaN,2019-02-28,ACA,NYSE,US25F1,NaN,NaN,NaN,\n Arcosa Inc \n (NYSE:ACA)\n
84648,1739445,"ARCOSA, INC.",2019-12-31,328795.0,31/12/2019,/2019/1739445_10K_2019_0001739445-20-000026.json,Yes,False,test,NaN,NaN,NaN,2020-02-27,ACA,NYSE,US25F1,NaN,NaN,NaN,\n Arcosa Inc \n (NYSE:ACA)\n
84649,1739445,"ARCOSA, INC.",2020-12-31,328795.0,31/12/2020,/2020/1739445_10K_2020_0001739445-21-000017.json,Yes,False,test,NaN,NaN,NaN,2021-02-25,ACA,NYSE,US25F1,NaN,NaN,NaN,\n Arcosa Inc \n (NYSE:ACA)\n
84650,1640147,SNOWFLAKE INC.,2021-01-31,339965.0,31/01/2021,/2021/1640147_10K_2021_0001640147-21-000073.json,Yes,False,test,NaN,NaN,NaN,2021-03-31,SNOW,NYSE,US28GL,NaN,NaN,NaN,\n Snowflake Inc \n (NYSE:SNOW)\n
84651,1822250,CONTEXTLOGIC INC.,2020-12-31,345980.0,31/12/2020,/2020/1822250_10K_2020_0001564590-21-015544.json,Yes,False,test,NaN,NaN,NaN,2021-03-25,WISH,NAS,US292W,NaN,NaN,NaN,\n ContextLogic Inc \n (NAS:WISH)\n


there is 233 that have different exchanges between first and second ticker

In [92]:
prediction_subset_copy = prediction_subset.copy()

companies_with_two_tickers_merged = prediction_subset_copy.merge(companies_with_two_tickers_matched, on='cik', how='left')

companies_with_two_tickers_merged.rename(columns={
    'ticker_x': 'ticker',
    'gurufocus-company-name_x': 'gurufocus-company-name',
    'company_x': 'company'
}, inplace=True)

# just for validation purpose
print(len(companies_with_two_tickers_merged['ticker_y'].unique())) # has Nan in it
print(len(companies_with_two_tickers_merged['cik'].unique()))

companies_with_two_tickers_merged.head(10)

491
9143


,cik,company,period_of_report,gvkey,datadate,filename,qualified,label,bankruptcy_prediction_split,bankruptcy_date_1,...,gurufocus-stockid,second-match-ticker,second-match-exchange,second-match-gurufocus-stockid,gurufocus-company-name,company_y,ticker_y,gurufocus_stockid,gurufocus-company-name_y,first_match
0,1750,AAR CORP,1994-05-31,1004.0,31/05/1994,/1994/1750_10K_1994_0000912057-94-002818.json,Yes,False,train,NaN,...,US06AR,NaN,NaN,NaN,\n AAR Corp \n (NYSE:AIR)\n,NaN,NaN,NaN,NaN,NaN
1,1750,AAR CORP,1995-05-31,1004.0,31/05/1995,/1995/1750_10K_1995_0000912057-95-006316.json,Yes,False,train,NaN,...,US06AR,NaN,NaN,NaN,\n AAR Corp \n (NYSE:AIR)\n,NaN,NaN,NaN,NaN,NaN
2,1750,AAR CORP,1996-05-31,1004.0,31/05/1996,/1996/1750_10K_1996_0000912057-96-018355.json,Yes,False,train,NaN,...,US06AR,NaN,NaN,NaN,\n AAR Corp \n (NYSE:AIR)\n,NaN,NaN,NaN,NaN,NaN
3,1750,AAR CORP,1997-05-31,1004.0,31/05/1997,/1997/1750_10K_1997_0000912057-97-028915.json,Yes,False,train,NaN,...,US06AR,NaN,NaN,NaN,\n AAR Corp \n (NYSE:AIR)\n,NaN,NaN,NaN,NaN,NaN
4,1750,AAR CORP,1998-05-31,1004.0,31/05/1998,/1998/1750_10K_1998_0001047469-98-032283.json,Yes,False,train,NaN,...,US06AR,NaN,NaN,NaN,\n AAR Corp \n (NYSE:AIR)\n,NaN,NaN,NaN,NaN,NaN
5,1750,AAR CORP,1999-05-31,1004.0,31/05/1999,/1999/1750_10K_1999_0001047469-99-032978.json,Yes,False,train,NaN,...,US06AR,NaN,NaN,NaN,\n AAR Corp \n (NYSE:AIR)\n,NaN,NaN,NaN,NaN,NaN
6,1750,AAR CORP,2000-05-31,1004.0,31/05/2000,/2000/1750_10K_2000_0000912057-00-039006.json,Yes,False,train,NaN,...,US06AR,NaN,NaN,NaN,\n AAR Corp \n (NYSE:AIR)\n,NaN,NaN,NaN,NaN,NaN
7,1750,AAR CORP,2001-05-31,1004.0,31/05/2001,/2001/1750_10K405_2001_0000912057-01-530303.json,Yes,False,train,NaN,...,US06AR,NaN,NaN,NaN,\n AAR Corp \n (NYSE:AIR)\n,NaN,NaN,NaN,NaN,NaN
8,1750,AAR CORP,2002-05-31,1004.0,31/05/2002,/2002/1750_10K_2002_0000912057-02-033450.json,Yes,False,train,NaN,...,US06AR,NaN,NaN,NaN,\n AAR Corp \n (NYSE:AIR)\n,NaN,NaN,NaN,NaN,NaN
9,1750,AAR CORP,2003-05-31,1004.0,31/05/2003,/2003/1750_10K_2003_0001047469-03-028005.json,Yes,False,train,NaN,...,US06AR,NaN,NaN,NaN,\n AAR Corp \n (NYSE:AIR)\n,NaN,NaN,NaN,NaN,NaN


In [95]:
mask = pd.notnull(companies_with_two_tickers_merged['ticker_y'])

companies_with_two_tickers_merged.loc[mask, 'ticker'] = companies_with_two_tickers_merged.loc[mask, 'ticker_y']
companies_with_two_tickers_merged.loc[mask, 'gurufocus-stockid'] = companies_with_two_tickers_merged.loc[mask, 'gurufocus_stockid']
companies_with_two_tickers_merged.loc[mask, 'gurufocus-company-name'] = companies_with_two_tickers_merged.loc[mask, 'gurufocus-company-name_y']

update_mask = (mask) & (companies_with_two_tickers_merged['first_match'] is False)
companies_with_two_tickers_merged.loc[update_mask, 'exchange'] = companies_with_two_tickers_merged.loc[update_mask, 'second-match-exchange']

In [99]:
len(companies_with_two_tickers_merged)

84652

In [100]:
companies_with_two_tickers_merged.drop(columns=['ticker_y', 'gurufocus-company-name_y', 'company_y', 'gurufocus_stockid', 'second-match-ticker', 'second-match-exchange', 'second-match-gurufocus-stockid'], inplace=True)

In [101]:
companies_with_two_tickers_merged.head(10)

,cik,company,period_of_report,gvkey,datadate,filename,qualified,label,bankruptcy_prediction_split,bankruptcy_date_1,bankruptcy_date_2,bankruptcy_date_3,filing_date,ticker,exchange,gurufocus-stockid,gurufocus-company-name,first_match
0,1750,AAR CORP,1994-05-31,1004.0,31/05/1994,/1994/1750_10K_1994_0000912057-94-002818.json,Yes,False,train,NaN,NaN,NaN,1994-08-24,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
1,1750,AAR CORP,1995-05-31,1004.0,31/05/1995,/1995/1750_10K_1995_0000912057-95-006316.json,Yes,False,train,NaN,NaN,NaN,1995-08-11,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
2,1750,AAR CORP,1996-05-31,1004.0,31/05/1996,/1996/1750_10K_1996_0000912057-96-018355.json,Yes,False,train,NaN,NaN,NaN,1996-08-20,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
3,1750,AAR CORP,1997-05-31,1004.0,31/05/1997,/1997/1750_10K_1997_0000912057-97-028915.json,Yes,False,train,NaN,NaN,NaN,1997-08-22,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
4,1750,AAR CORP,1998-05-31,1004.0,31/05/1998,/1998/1750_10K_1998_0001047469-98-032283.json,Yes,False,train,NaN,NaN,NaN,1998-08-20,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
5,1750,AAR CORP,1999-05-31,1004.0,31/05/1999,/1999/1750_10K_1999_0001047469-99-032978.json,Yes,False,train,NaN,NaN,NaN,1999-08-18,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
6,1750,AAR CORP,2000-05-31,1004.0,31/05/2000,/2000/1750_10K_2000_0000912057-00-039006.json,Yes,False,train,NaN,NaN,NaN,2000-08-24,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
7,1750,AAR CORP,2001-05-31,1004.0,31/05/2001,/2001/1750_10K405_2001_0000912057-01-530303.json,Yes,False,train,NaN,NaN,NaN,2001-08-27,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
8,1750,AAR CORP,2002-05-31,1004.0,31/05/2002,/2002/1750_10K_2002_0000912057-02-033450.json,Yes,False,train,NaN,NaN,NaN,2002-08-26,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
9,1750,AAR CORP,2003-05-31,1004.0,31/05/2003,/2003/1750_10K_2003_0001047469-03-028005.json,Yes,False,train,NaN,NaN,NaN,2003-08-18,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN


In [102]:
datetime_now = get_datetime_now()
companies_with_two_tickers_merged.to_csv(f'ECL_with_ticker_{datetime_now}.csv')

#### processing companies for which cik was not found on gurufocus site

In [113]:
not_found_gurufocus_ciks_two_ticker_case_df = pd.DataFrame(not_found_gurufocus_ciks_two_ticker_case, columns=['cik', 'gurufocus_stockid', 'ticker'])
not_found_gurufocus_ciks_two_ticker_case_df.head(30)

,cik,gurufocus_stockid,ticker
0,1004980,CA03R4,PGN.H
1,1016933,CA01JL,R
2,1061068,CA02XP,GBO.H
3,1129989,CA03R4,PGN.H
4,1143155,US0ST1,XBKS
5,1387467,CA00WV,HAP.UN
6,1393311,CA01I2,PUB
7,1442741,US0ST1,XBKS
8,1600125,US04DV,MRDN
9,1656239,US0STA,PNK


half of the companies have invalid match in the result file 'ECL_with_ticker_2024-04-19_12-05-32.csv'
#### mayby post check will filter them out

# Processing the companies with one ticker

In [139]:
prediction_subset = pd.read_csv('ECL_with_ticker_2024-04-19_12-05-32.csv', index_col=0)
len(prediction_subset)

C:\Users\grzeg\AppData\Local\Temp\ipykernel_12468\1259041293.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  prediction_subset = pd.read_csv('ECL_with_ticker_2024-04-19_12-05-32.csv', index_col=0)


84652

In [140]:
prediction_subset.head(10)

,cik,company,period_of_report,gvkey,datadate,filename,qualified,label,bankruptcy_prediction_split,bankruptcy_date_1,bankruptcy_date_2,bankruptcy_date_3,filing_date,ticker,exchange,gurufocus-stockid,gurufocus-company-name,first_match
0,1750,AAR CORP,1994-05-31,1004.0,31/05/1994,/1994/1750_10K_1994_0000912057-94-002818.json,Yes,False,train,NaN,NaN,NaN,1994-08-24,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
1,1750,AAR CORP,1995-05-31,1004.0,31/05/1995,/1995/1750_10K_1995_0000912057-95-006316.json,Yes,False,train,NaN,NaN,NaN,1995-08-11,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
2,1750,AAR CORP,1996-05-31,1004.0,31/05/1996,/1996/1750_10K_1996_0000912057-96-018355.json,Yes,False,train,NaN,NaN,NaN,1996-08-20,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
3,1750,AAR CORP,1997-05-31,1004.0,31/05/1997,/1997/1750_10K_1997_0000912057-97-028915.json,Yes,False,train,NaN,NaN,NaN,1997-08-22,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
4,1750,AAR CORP,1998-05-31,1004.0,31/05/1998,/1998/1750_10K_1998_0001047469-98-032283.json,Yes,False,train,NaN,NaN,NaN,1998-08-20,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
5,1750,AAR CORP,1999-05-31,1004.0,31/05/1999,/1999/1750_10K_1999_0001047469-99-032978.json,Yes,False,train,NaN,NaN,NaN,1999-08-18,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
6,1750,AAR CORP,2000-05-31,1004.0,31/05/2000,/2000/1750_10K_2000_0000912057-00-039006.json,Yes,False,train,NaN,NaN,NaN,2000-08-24,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
7,1750,AAR CORP,2001-05-31,1004.0,31/05/2001,/2001/1750_10K405_2001_0000912057-01-530303.json,Yes,False,train,NaN,NaN,NaN,2001-08-27,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
8,1750,AAR CORP,2002-05-31,1004.0,31/05/2002,/2002/1750_10K_2002_0000912057-02-033450.json,Yes,False,train,NaN,NaN,NaN,2002-08-26,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN
9,1750,AAR CORP,2003-05-31,1004.0,31/05/2003,/2003/1750_10K_2003_0001047469-03-028005.json,Yes,False,train,NaN,NaN,NaN,2003-08-18,AIR,NYSE,US06AR,\n AAR Corp \n (NYSE:AIR)\n,NaN


In [141]:
prediction_subset['gurufocus-company-name'] = clean_gurufocus_company_name_column(prediction_subset['gurufocus-company-name'])

prediction_subset.tail(10)

,cik,company,period_of_report,gvkey,datadate,filename,qualified,label,bankruptcy_prediction_split,bankruptcy_date_1,bankruptcy_date_2,bankruptcy_date_3,filing_date,ticker,exchange,gurufocus-stockid,gurufocus-company-name,first_match
84642,1796209,API GROUP CORP,2020-12-31,325576.0,31/12/2020,/2020/1796209_10K_2020_0001564590-21-014986.json,Yes,False,test,NaN,NaN,NaN,2021-03-24,APG,NYSE,US0UBH,APi Group Corp,NaN
84643,1810739,"RADIUS GLOBAL INFRASTRUCTURE, INC.",2020-12-31,325942.0,31/12/2020,/2020/1810739_10K_2020_0001564590-21-016621.json,Yes,False,test,NaN,NaN,NaN,2021-03-30,RADI,NAS,US28N7,Radius Global Infrastructure Inc (Delisted),NaN
84644,1720635,NVENT ELECTRIC PLC,2018-12-31,326688.0,31/12/2018,/2018/1720635_10K_2018_0001720635-19-000008.json,Yes,False,test,NaN,NaN,NaN,2019-02-19,NVT,NYSE,US0V7E,nVent Electric PLC,NaN
84645,1720635,NVENT ELECTRIC PLC,2019-12-31,326688.0,31/12/2019,/2019/1720635_10K_2019_0001720635-20-000009.json,Yes,False,test,NaN,NaN,NaN,2020-02-25,NVT,NYSE,US0V7E,nVent Electric PLC,NaN
84646,1720635,NVENT ELECTRIC PLC,2020-12-31,326688.0,31/12/2020,/2020/1720635_10K_2020_0001720635-21-000013.json,Yes,False,test,NaN,NaN,NaN,2021-02-23,NVT,NYSE,US0V7E,nVent Electric PLC,NaN
84647,1739445,"ARCOSA, INC.",2018-12-31,328795.0,31/12/2018,/2018/1739445_10K_2018_0001739445-19-000020.json,Yes,False,test,NaN,NaN,NaN,2019-02-28,ACA,NYSE,US25F1,Arcosa Inc,NaN
84648,1739445,"ARCOSA, INC.",2019-12-31,328795.0,31/12/2019,/2019/1739445_10K_2019_0001739445-20-000026.json,Yes,False,test,NaN,NaN,NaN,2020-02-27,ACA,NYSE,US25F1,Arcosa Inc,NaN
84649,1739445,"ARCOSA, INC.",2020-12-31,328795.0,31/12/2020,/2020/1739445_10K_2020_0001739445-21-000017.json,Yes,False,test,NaN,NaN,NaN,2021-02-25,ACA,NYSE,US25F1,Arcosa Inc,NaN
84650,1640147,SNOWFLAKE INC.,2021-01-31,339965.0,31/01/2021,/2021/1640147_10K_2021_0001640147-21-000073.json,Yes,False,test,NaN,NaN,NaN,2021-03-31,SNOW,NYSE,US28GL,Snowflake Inc,NaN
84651,1822250,CONTEXTLOGIC INC.,2020-12-31,345980.0,31/12/2020,/2020/1822250_10K_2020_0001564590-21-015544.json,Yes,False,test,NaN,NaN,NaN,2021-03-25,WISH,NAS,US292W,ContextLogic Inc,NaN


In [142]:
datetime_now = get_datetime_now()
prediction_subset.to_csv(f'ECL_with_ticker_{datetime_now}.csv')

In [143]:
prediction_subset_grouped = prediction_subset.groupby('cik').agg({
    'ticker': 'last',
    'company': 'last',
    'gurufocus-company-name': 'last',
    'gurufocus-stockid': 'last'
}).reset_index()
prediction_subset_grouped.head(10)

,cik,ticker,company,gurufocus-company-name,gurufocus-stockid
0,1750,AIR,AAR CORP,AAR Corp,US06AR
1,1800,ABT,ABBOTT LABORATORIES,Abbott Laboratories,US066X
2,2024,None,ACE HARDWARE CORP,nan,None
3,2034,None,ACETO CORP,nan,None
4,2135,None,AFFILIATED COMPUTER SERVICES INC,nan,None
5,2178,None,"ADAMS RESOURCES & ENERGY, INC.",nan,None
6,2488,AMD,ADVANCED MICRO DEVICES INC,Advanced Micro Devices Inc,US022E
7,2491,BYI,"BALLY TECHNOLOGIES, INC.",Bally Technologies Inc (Delisted),US06R5
8,2601,ARXX,AEROFLEX INC,Aeroflex Inc (Delisted),US026H
9,2852,None,AGWAY INC,nan,None


In [144]:
import os

directory = 'gurufocus-immediate-response-for-stockid'

not_found_gurufocus_ciks = []

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        
        with open(f'{directory}/{filename}', 'r', encoding='utf-8') as f:
            content = f.read()
            cik_index = content.find('cik')  
            
            if cik_index != -1:
                
                filename = filename.replace('.txt', '')
                filename = filename.replace('page_source-', '')
                cik, gurufocus_stockid = filename.split('_')[0].split('-')
                ticker = filename.split('_')[1]
                
                gurufocus_cik = content[cik_index + 5: cik_index + 15]
                try:
                    gurufocus_cik = int(gurufocus_cik.split('"')[0])
                    mask = (prediction_subset_grouped['cik'] == int(cik)) & (prediction_subset_grouped['gurufocus-stockid'] == gurufocus_stockid)
                    prediction_subset_grouped.loc[mask, 'gurufocus-cik'] = gurufocus_cik
                except Exception as ex:
                    print(f"Error for file: {filename}. Gurufocus cik: {gurufocus_cik}")
                    not_found_gurufocus_ciks.append([cik, gurufocus_stockid, ticker])
                
            else:
                print(f"cik not found. File: {filename}")
            
    else:
        print(f"File {filename} is not a TXT file")

Error for file: 1009918-CA03G9_MNC.H. Gurufocus cik: ,class_des
Error for file: 1024645-CA01WC_TTN.A. Gurufocus cik: ,class_des
Error for file: 1088825-US0ST2_TIVO. Gurufocus cik: ,class_des
Error for file: 1254699-CA03VM_QVC. Gurufocus cik: ,class_des
Error for file: 1383571-US0STE_GSM. Gurufocus cik: ,class_des
Error for file: 1469510-US09AF_REN. Gurufocus cik: ,class_des
Error for file: 1509589-US06K2_BCEI. Gurufocus cik: ,class_des
Error for file: 1575988-US0ST4_EAGL. Gurufocus cik: ,class_des
Error for file: 20171-US0ST6_CB. Gurufocus cik: ,class_des
Error for file: 20520-US03FQ_FTRPR.PFD. Gurufocus cik: ,class_des
Error for file: 29646-US0712_CRPCL.PFD. Gurufocus cik: ,class_des
Error for file: 36204-US0AHH_FCMM. Gurufocus cik: ,class_des
Error for file: 704051-US08AK_LM. Gurufocus cik: ,class_des
Error for file: 729237-US0ST0_PKY. Gurufocus cik: ,class_des
Error for file: 916530-US07CM_ENN. Gurufocus cik: ,class_des
Error for file: 931581-CA03A3_KTI. Gurufocus cik: ,class_des


In [145]:
len(prediction_subset_grouped[prediction_subset_grouped['gurufocus-cik'].notnull()])

5055

In [147]:
prediction_subset_grouped = prediction_subset_grouped[prediction_subset_grouped['gurufocus-cik'].notnull()]

In [148]:
prediction_subset_grouped['cik-equal-gurufocus-cik'] = prediction_subset_grouped['cik'] == prediction_subset_grouped['gurufocus-cik']

prediction_subset_grouped['cik-equal-gurufocus-cik'].value_counts()

cik-equal-gurufocus-cik
True     4925
False     130
Name: count, dtype: int64

In [149]:
prediction_subset_grouped_read_to_merge = prediction_subset_grouped.drop(columns=['ticker', 'company', 'gurufocus-cik', 'gurufocus-company-name', 'gurufocus-stockid'])
prediction_subset_grouped_read_to_merge.head(10)

,cik,cik-equal-gurufocus-cik
1,1800,True
6,2488,True
7,2491,True
8,2601,True
10,2969,True
16,3197,True
19,3370,True
21,3453,True
23,3545,True
24,3570,True


In [153]:
companies_merged = prediction_subset.merge(prediction_subset_grouped_read_to_merge, on='cik', how='left')

print(len(companies_merged))
companies_merged.head(10)

84652


,cik,company,period_of_report,gvkey,datadate,filename,qualified,label,bankruptcy_prediction_split,bankruptcy_date_1,bankruptcy_date_2,bankruptcy_date_3,filing_date,ticker,exchange,gurufocus-stockid,gurufocus-company-name,first_match,cik-equal-gurufocus-cik
0,1750,AAR CORP,1994-05-31,1004.0,31/05/1994,/1994/1750_10K_1994_0000912057-94-002818.json,Yes,False,train,NaN,NaN,NaN,1994-08-24,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
1,1750,AAR CORP,1995-05-31,1004.0,31/05/1995,/1995/1750_10K_1995_0000912057-95-006316.json,Yes,False,train,NaN,NaN,NaN,1995-08-11,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
2,1750,AAR CORP,1996-05-31,1004.0,31/05/1996,/1996/1750_10K_1996_0000912057-96-018355.json,Yes,False,train,NaN,NaN,NaN,1996-08-20,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
3,1750,AAR CORP,1997-05-31,1004.0,31/05/1997,/1997/1750_10K_1997_0000912057-97-028915.json,Yes,False,train,NaN,NaN,NaN,1997-08-22,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
4,1750,AAR CORP,1998-05-31,1004.0,31/05/1998,/1998/1750_10K_1998_0001047469-98-032283.json,Yes,False,train,NaN,NaN,NaN,1998-08-20,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
5,1750,AAR CORP,1999-05-31,1004.0,31/05/1999,/1999/1750_10K_1999_0001047469-99-032978.json,Yes,False,train,NaN,NaN,NaN,1999-08-18,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
6,1750,AAR CORP,2000-05-31,1004.0,31/05/2000,/2000/1750_10K_2000_0000912057-00-039006.json,Yes,False,train,NaN,NaN,NaN,2000-08-24,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
7,1750,AAR CORP,2001-05-31,1004.0,31/05/2001,/2001/1750_10K405_2001_0000912057-01-530303.json,Yes,False,train,NaN,NaN,NaN,2001-08-27,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
8,1750,AAR CORP,2002-05-31,1004.0,31/05/2002,/2002/1750_10K_2002_0000912057-02-033450.json,Yes,False,train,NaN,NaN,NaN,2002-08-26,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
9,1750,AAR CORP,2003-05-31,1004.0,31/05/2003,/2003/1750_10K_2003_0001047469-03-028005.json,Yes,False,train,NaN,NaN,NaN,2003-08-18,AIR,NYSE,US06AR,AAR Corp,NaN,NaN


In [156]:
companies_merged['cik-equal-gurufocus-cik'].value_counts()

cik-equal-gurufocus-cik
True     52728
False      818
Name: count, dtype: int64

In [165]:
companies_merged_grouped = companies_merged.groupby('cik').agg({
    'ticker': 'last',
    'company': 'last',
    'gurufocus-company-name': 'last',
    'gurufocus-stockid': 'last',
    'cik-equal-gurufocus-cik': 'last',
    'first_match': 'last'
}).reset_index()

In [166]:
companies_merged_grouped['cik-equal-gurufocus-cik'].value_counts()

cik-equal-gurufocus-cik
True     4925
False     130
Name: count, dtype: int64

In [167]:
companies_merged_grouped['first_match'].isnull().sum()

8651

In [172]:
print(companies_merged['cik-equal-gurufocus-cik'].notnull().sum())
print(companies_merged['first_match'].isnull().sum())
print(len(companies_merged))

53546
77582
84652


In [173]:
mask = companies_merged['first_match'].notnull()
companies_merged.loc[mask, 'cik-equal-gurufocus-cik'] = True

In [174]:
print(companies_merged['cik-equal-gurufocus-cik'].notnull().sum())
print(companies_merged['first_match'].isnull().sum())
print(len(companies_merged))

60616
77582
84652


In [175]:
companies_merged.head(10)

,cik,company,period_of_report,gvkey,datadate,filename,qualified,label,bankruptcy_prediction_split,bankruptcy_date_1,bankruptcy_date_2,bankruptcy_date_3,filing_date,ticker,exchange,gurufocus-stockid,gurufocus-company-name,first_match,cik-equal-gurufocus-cik
0,1750,AAR CORP,1994-05-31,1004.0,31/05/1994,/1994/1750_10K_1994_0000912057-94-002818.json,Yes,False,train,NaN,NaN,NaN,1994-08-24,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
1,1750,AAR CORP,1995-05-31,1004.0,31/05/1995,/1995/1750_10K_1995_0000912057-95-006316.json,Yes,False,train,NaN,NaN,NaN,1995-08-11,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
2,1750,AAR CORP,1996-05-31,1004.0,31/05/1996,/1996/1750_10K_1996_0000912057-96-018355.json,Yes,False,train,NaN,NaN,NaN,1996-08-20,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
3,1750,AAR CORP,1997-05-31,1004.0,31/05/1997,/1997/1750_10K_1997_0000912057-97-028915.json,Yes,False,train,NaN,NaN,NaN,1997-08-22,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
4,1750,AAR CORP,1998-05-31,1004.0,31/05/1998,/1998/1750_10K_1998_0001047469-98-032283.json,Yes,False,train,NaN,NaN,NaN,1998-08-20,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
5,1750,AAR CORP,1999-05-31,1004.0,31/05/1999,/1999/1750_10K_1999_0001047469-99-032978.json,Yes,False,train,NaN,NaN,NaN,1999-08-18,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
6,1750,AAR CORP,2000-05-31,1004.0,31/05/2000,/2000/1750_10K_2000_0000912057-00-039006.json,Yes,False,train,NaN,NaN,NaN,2000-08-24,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
7,1750,AAR CORP,2001-05-31,1004.0,31/05/2001,/2001/1750_10K405_2001_0000912057-01-530303.json,Yes,False,train,NaN,NaN,NaN,2001-08-27,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
8,1750,AAR CORP,2002-05-31,1004.0,31/05/2002,/2002/1750_10K_2002_0000912057-02-033450.json,Yes,False,train,NaN,NaN,NaN,2002-08-26,AIR,NYSE,US06AR,AAR Corp,NaN,NaN
9,1750,AAR CORP,2003-05-31,1004.0,31/05/2003,/2003/1750_10K_2003_0001047469-03-028005.json,Yes,False,train,NaN,NaN,NaN,2003-08-18,AIR,NYSE,US06AR,AAR Corp,NaN,NaN


In [176]:
datetime_now = get_datetime_now()
companies_merged.to_csv(f'ECL_with_ticker_{datetime_now}.csv')

In [177]:
companies_merged['cik-equal-gurufocus-cik'].value_counts()

cik-equal-gurufocus-cik
True     59798
False      818
Name: count, dtype: int64

In [2]:
import pandas as pd
companies_merged = pd.read_csv('ECL_with_ticker_2024-04-19_14-35-21.csv', index_col=0)

C:\Users\grzeg\AppData\Local\Temp\ipykernel_25804\308524012.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  companies_merged = pd.read_csv('ECL_with_ticker_2024-04-19_14-35-21.csv', index_col=0)


In [3]:
companies_merged_eligible = companies_merged[companies_merged['cik-equal-gurufocus-cik'] == True]

print(len(companies_merged_eligible))

59798


also insert true where first_match is set

In [207]:
companies_merged_eligible_grouped = companies_merged_eligible.groupby('cik').agg({
    'company': 'last',
    'label': 'last',
    'ticker': 'last',
    'gurufocus-company-name': 'last',
    'gurufocus-stockid': 'last',
    'filing_date': 'last',
    'cik-equal-gurufocus-cik': 'last',
    'first_match': 'last'
}).reset_index()

print(len(companies_merged_eligible_grouped))

5417


In [194]:
companies_merged_eligible_grouped.head(10)

,cik,company,label,ticker,gurufocus-company-name,gurufocus-stockid,filing_date,cik-equal-gurufocus-cik,first_match
0,1800,ABBOTT LABORATORIES,False,ABT,Abbott Laboratories,US066X,2021-02-19,True,None
1,2488,ADVANCED MICRO DEVICES INC,False,AMD,Advanced Micro Devices Inc,US022E,2021-01-29,True,None
2,2491,"BALLY TECHNOLOGIES, INC.",False,BYI,Bally Technologies Inc (Delisted),US06R5,2014-08-29,True,None
3,2601,AEROFLEX INC,False,ARXX,Aeroflex Inc (Delisted),US026H,2006-09-13,True,None
4,2969,AIR PRODUCTS & CHEMICALS INC /DE/,False,APD,Air Products & Chemicals Inc,US06DU,2021-11-18,True,None
5,3153,ALABAMA POWER CO,False,ALPpQ.PFD,Alabama Power Co (Delisted),US0TUZ,2021-02-18,True,True
6,3197,CECO ENVIRONMENTAL CORP,False,CECO,CECO Environmental Corp,US02K6,2021-03-03,True,None
7,3370,IKON OFFICE SOLUTIONS INC,False,IKN,Ikon Office Solutions Inc (Delisted),US07ZK,2007-11-29,True,None
8,3449,ALEXANDER & ALEXANDER SERVICES INC,False,AAL,Alexander & Alexander (Delisted),US066A,1996-03-29,True,False
9,3453,"MATSON, INC.",False,MATX,Matson Inc,US08CS,2021-02-26,True,None


In [254]:
import re
pattern = re.compile(r"[\\/.,\"]") 

def clear_company_name(company_name):
    company_name = company_name.lower()
    company_name = re.sub(r"[\\/(].*", '', company_name)
    company_name = pattern.sub('', company_name)
    company_name = company_name.strip()
    company_name = re.sub(r'^the ', '', company_name)
    company_name = company_name.replace(' ', '')
    company_name = company_name.replace('&', 'and')
    return company_name

In [255]:
clear_company_name('The ALLIED PRODUCTS CORP /DE/')

'alliedproductscorp'

In [256]:
import re

def calc_strings_similarity(str1, str2):
    str1 = clear_company_name(str1)
    str2 = clear_company_name(str2)
    
    shorter_str = str1 if len(str1) < len(str2) else str2
    match_counter = 0
    length = len(shorter_str)
    for i in range(length):
        if str1[i] == str2[i]:
            match_counter += 1
    if length == 0:
        return 0
    return match_counter / length

In [189]:
datetime_now = get_datetime_now()
companies_merged_eligible_grouped.to_csv(f'companies_eligible_to_financial_dataset_{datetime_now}.csv')

In [252]:
calc_strings_similarity('Allied Products Corp  (Delisted)', 'The ALLIED PRODUCTS CORP /DE/')

1.0

In [257]:
companies_merged_eligible_grouped_copy = companies_merged_eligible_grouped.copy()

companies_merged_eligible_grouped_copy['similarity'] = companies_merged_eligible_grouped_copy.apply(lambda x: calc_strings_similarity(x['company'], x['gurufocus-company-name']), axis=1)

not_similar_companies = companies_merged_eligible_grouped_copy[companies_merged_eligible_grouped_copy['similarity'] < 0.6]

print(len(not_similar_companies))
not_similar_companies.head()

4


,cik,company,label,ticker,gurufocus-company-name,gurufocus-stockid,filing_date,cik-equal-gurufocus-cik,first_match,similarity
702,316206,"STARWOOD HOTEL & RESORTS WORLDWIDE, INC",False,HOT,Starwood Hotels & Resorts Worldwide Inc (Deli...,US07VK,2016-02-25,True,None,0.371429
2843,1062231,AMERICAN AXLE & MANUFACTURING HOLDINGS INC,False,AXL,American Axle & Mfg Holdings Inc,US06HP,2021-02-12,True,None,0.551724
3462,1177394,SYNNEX CORP,False,SNX,TD Synnex Corp,US09JE,2021-01-28,True,None,0.000000
4250,1437107,"DISCOVERY, INC.",False,WBD,Warner Bros. Discovery Inc,US02YW,2021-02-22,True,True,0.083333
